<a href="https://colab.research.google.com/github/evanekawijaya/data-project/blob/main/DAS%202021/regency_cluster_(structured_explanations).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#importing the data and initial data pre-processing
import pandas as pd
from google.colab import drive

drive.mount('/content/drive')
train=pd.read_csv("/content/drive/MyDrive/DAS/Train.csv").iloc[:,2:]
test=pd.read_csv("/content/drive/MyDrive/DAS/Test.csv").iloc[:,3:]

train = train.drop(columns=['distance','buying_date','dealing_date','cnt'])
train = train.astype('category')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
#variable selection
X = train.drop(columns=['regency_cluster'])
y = train['regency_cluster']

from sklearn.feature_selection import SelectKBest, chi2
X_new = SelectKBest(chi2, k=10).fit_transform(X,y)
X_new[0:2]

array([[    2,    66,   348, 48862,    12,     1,  8250,     1,     2,
           50],
       [    2,    66,   348, 48862,    12,     1,  8250,     1,     2,
           50]])

In [4]:
#dropping the unselected variables
drop=['continent_id','buyer_country','buyer_region','mobile','channel_id','adults','children','room','dealing','regency_continent']
train=train.drop(columns=drop)

X = train.drop(columns=['regency_cluster'])
y = train['regency_cluster']

In [5]:
#modelling and validation
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold

model = RandomForestClassifier(max_samples=0.4,max_features=5,n_estimators=50)
cv = RepeatedStratifiedKFold(n_splits=10,n_repeats=3, random_state=2056)
n_scores = cross_val_score(model, X, y, cv=cv,scoring='f1_micro', error_score='raise')
print('f1 score: %.3f (%.3f)' % (np.mean(n_scores), np.std(n_scores)))

f1 score: 0.312 (0.003)


In [9]:
#data pre-processing for the test dataset
drop1=['continent_id','buyer_country','buyer_region','distance','mobile','buying_date','dealing_date','channel_id','adults','children','room','regency_continent']
test=test.drop(columns=drop1).astype('category')

In [10]:
#model fitting and predicting the test dataset
model.fit(X,y)
pred = model.predict(test)
answer = test
answer['regency_cluster'] = pred

In [17]:
from google.colab import files
answer.to_csv('ID-21-0250_Siev_Universitas Indonesia.csv',index=False)
files.download('ID-21-0250_Siev_Universitas Indonesia.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [6]:
#finding the best number of samples
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2, stratify=y,random_state= 2056)
for i in np.arange(0.1,1.1,0.1):
  arr=[]
  if i==1.0:
    i=None
  for j in range(0,5):
    model = RandomForestClassifier(max_samples=i)
    model.fit(X_train, y_train)
    yhat = model.predict(X_test)
    # evaluate predictions
    arr.append(f1_score(y_test, yhat,average='micro'))
  print(i,"mean:{0:.3f}, std:{1:.3f}".format(np.mean(arr), np.std(arr)))

NameError: ignored

In [ ]:
#finding the best number of trees
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2, stratify=y,random_state= 2056)
for i in [50,100]:
  model = RandomForestClassifier(max_samples=0.4,n_estimators=i)
  model.fit(X_train, y_train)
  yhat = model.predict(X_test)
  f1=f1_score(y_test, yhat,average='micro')
  print(i,"score:{0:.3f}".format(f1))

In [ ]:
#finding the best number of features
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2, stratify=y,random_state= 2056)
for i in range(1,len(X.columns)+1):
  arr=[]
  for j in range(0,5):
    model = RandomForestClassifier(max_samples=0.4,max_features=i,n_estimators=50)
    model.fit(X_train, y_train)
    yhat = model.predict(X_test)
    # evaluate predictions
    arr.append(f1_score(y_test, yhat,average='micro'))
  print(i,"mean:{0:.3f}, std:{1:.3f}".format(np.mean(arr), np.std(arr)))